In [1]:
import os

import numpy as np
import pandas as pd
from scipy.stats import linregress

# === Step 1: Load OHLCV close prices ===
input_path = "../data/BTCUSDT_1min.csv"
df_btc = pd.read_csv(input_path, parse_dates=["timestamp"])
df_prices = df_btc[['timestamp', 'close']].copy()
df_prices.set_index("timestamp", inplace=True)

# === Step 2: Extract 12-hour windows with 1-hour step ===
window_size = 720  # 12 hours of 1-min bars
step_size = 120     # 1 hour step
window_centers, regimes, vols, slopes = [], [], [], []

for i in range(0, len(df_prices) - window_size, step_size):
    window_df = df_prices.iloc[i:i + window_size]
    center_time = window_df.index[window_size // 2]

    avg_price = window_df["close"].values
    x = np.arange(len(avg_price))
    slope = linregress(x, avg_price).slope
    vol = window_df['close'].pct_change().dropna().std()

    window_centers.append(center_time)
    slopes.append(slope)
    vols.append(vol)

# === Step 3: Define thresholds ===
vol_thresh = np.median(vols)
slope_thresh = np.percentile(np.abs(slopes), 10)

# === Step 4: Assign regimes ===
for slope, vol in zip(slopes, vols):
    if abs(slope) < slope_thresh:
        regimes.append("Sideways")
    elif slope > 0 and vol > vol_thresh:
        regimes.append("HighVol_UpTrend")
    elif slope > 0:
        regimes.append("LowVol_UpTrend")
    elif slope < 0 and vol > vol_thresh:
        regimes.append("HighVol_DownTrend")
    else:
        regimes.append("LowVol_DownTrend")

# === Step 5: Tag each fragment ===
fragments = []
for i, center_time in enumerate(window_centers):
    label = regimes[i]
    start, end = i * step_size, i * step_size + window_size
    window_df = df_btc.iloc[start:end].copy()
    window_df["regime_name"] = label
    fragments.append(window_df)

df_fragments = pd.concat(fragments)
df_fragments.set_index("timestamp", inplace=True)

# === Step 6: Save regime-specific CSVs ===
output_dir = "experiments/data_experiment/regime_fragments"
os.makedirs(output_dir, exist_ok=True)

output_paths = {}
for label in sorted(df_fragments["regime_name"].unique()):
    df_sub = df_fragments[df_fragments["regime_name"] == label]
    file_name = f"fragment_data_{label}.csv"
    full_path = os.path.join(output_dir, file_name)
    df_sub.to_csv(full_path)
    output_paths[label] = full_path

# === Debug Info ===
print("\nSaved 12-hours regime fragments:")
for regime, path in output_paths.items():
    print(f"{regime:25s} → {path}")


Saved 12-hours regime fragments:
HighVol_DownTrend         → experiments/data_experiment/regime_fragments\fragment_data_HighVol_DownTrend.csv
HighVol_UpTrend           → experiments/data_experiment/regime_fragments\fragment_data_HighVol_UpTrend.csv
LowVol_DownTrend          → experiments/data_experiment/regime_fragments\fragment_data_LowVol_DownTrend.csv
LowVol_UpTrend            → experiments/data_experiment/regime_fragments\fragment_data_LowVol_UpTrend.csv
Sideways                  → experiments/data_experiment/regime_fragments\fragment_data_Sideways.csv


In [2]:
import pandas as pd
import numpy as np
import os
from scipy.stats import linregress

# === Config ===
regime_dir = "experiments/data_experiment/regime_fragments"  # Path to your 12-hour regime fragments
output_dir = "experiments/backtest_fragments"
os.makedirs(output_dir, exist_ok=True)

fragment_length = 720  # 12 hours = 720 minutes of 1-min bars

# === Load all fragments ===
fragment_dfs = []
for file in os.listdir(regime_dir):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(regime_dir, file), parse_dates=["timestamp"])
        regime = file.replace("fragment_data_", "").replace(".csv", "")
        df["regime_name"] = regime
        df["fragment_id"] = np.repeat(np.arange(len(df) // fragment_length), fragment_length)
        fragment_dfs.append(df)

df_all = pd.concat(fragment_dfs)

# === Compute slope and volatility ===
features = []
for (regime, fid), group in df_all.groupby(["regime_name", "fragment_id"]):
    if len(group) < fragment_length:
        continue
    x = np.arange(len(group))
    y = group["close"].values
    slope = linregress(x, y).slope
    volatility = group["close"].pct_change().std()

    features.append({
        "regime_name": regime,
        "fragment_id": fid,
        "slope": slope,
        "slope_abs": abs(slope),
        "volatility": volatility
    })

df_feat = pd.DataFrame(features)

# === Normalize and score ===
df_feat["vol_norm"] = df_feat.groupby("regime_name")["volatility"].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
df_feat["slope_norm"] = df_feat.groupby("regime_name")["slope_abs"].transform(lambda x: (x - x.min()) / (x.max() - x.min()))

def score(row):
    if row["regime_name"] == "Sideways":
        return (1 - row["slope_norm"]) * (1 - row["vol_norm"])
    elif "UpTrend" in row["regime_name"]:
        return row["slope_norm"] * (1 - row["vol_norm"]) if "LowVol" in row["regime_name"] else row["slope_norm"] * row["vol_norm"]
    elif "DownTrend" in row["regime_name"]:
        return row["slope_norm"] * (1 - row["vol_norm"]) if "LowVol" in row["regime_name"] else row["slope_norm"] * row["vol_norm"]
    else:
        return 0.0

df_feat["score"] = df_feat.apply(score, axis=1)

# === Select best fragment per regime ===
best_fragments = df_feat.sort_values("score", ascending=False).groupby("regime_name").first().reset_index()

# === Extract and save best fragments ===
selected_dfs = []
for _, row in best_fragments.iterrows():
    regime = row["regime_name"]
    fid = row["fragment_id"]
    path = os.path.join(regime_dir, f"fragment_data_{regime}.csv")
    df = pd.read_csv(path, parse_dates=["timestamp"])
    df["fragment_id"] = np.repeat(np.arange(len(df) // fragment_length), fragment_length)
    df_best = df[df["fragment_id"] == fid].copy()
    selected_dfs.append(df_best)

df_final = pd.concat(selected_dfs).sort_values("timestamp")
output_path = os.path.join(output_dir, "best_fragments_combined.csv")
df_final.to_csv(output_path, index=False)

print(f"[Save] Saved best 12-hour fragments per regime → {output_path}")
print("[Info] Regimes found:", df_final['regime_name'].unique())

[Save] Saved best 12-hour fragments per regime → experiments/backtest_fragments\best_fragments_combined.csv
[Info] Regimes found: ['HighVol_DownTrend' 'HighVol_UpTrend' 'Sideways' 'LowVol_DownTrend'
 'LowVol_UpTrend']


In [3]:
import pandas as pd
import plotly.graph_objects as go
import os

# === Load best regime fragments ===
csv_path = os.path.join("experiments", "backtest_fragments", "best_fragments_combined.csv")
df = pd.read_csv(csv_path, parse_dates=["timestamp"])
df.set_index("timestamp", inplace=True)

# === Sort by time, then group by regime ===
df = df.sort_index()
regimes = df["regime_name"].unique()

# === Limit to top 5 regimes (or less if fewer exist) ===
top_regimes = list(regimes[:5])

for regime in top_regimes:
    df_plot = df[df["regime_name"] == regime].copy()

    # === Optional: Clamp outlier highs/lows for better visual ===
    q1 = df_plot["close"].quantile(0.25)
    q3 = df_plot["close"].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 3 * iqr
    upper = q3 + 3 * iqr
    df_plot["high"] = df_plot["high"].clip(upper=upper)
    df_plot["low"] = df_plot["low"].clip(lower=lower)

    # === Plot ===
    fig = go.Figure()

    # Candlestick
    fig.add_trace(go.Candlestick(
        x=df_plot.index,
        open=df_plot["open"],
        high=df_plot["high"],
        low=df_plot["low"],
        close=df_plot["close"],
        name="Price",
        increasing_line_color='lime',
        decreasing_line_color='red'
    ))

    # Layout
    fig.update_layout(
        title=f"Best 12-Hour Fragment – {regime}",
        xaxis_title="Time",
        yaxis_title="Price",
        yaxis2=dict(overlaying='y', side='right', showgrid=False, title='Volume'),
        height=600,
        xaxis_rangeslider_visible=False,
        template="plotly_white"
    )

    fig.show()

In [4]:
import pandas as pd
from signals.model_preparation import prepare_features_only
from train_model.signal_model import SignalModel
from train_model.vol_model import VolatilityModel
from logs.prediction_logger import PredictionLogger
from logs.signal_logger import SignalHistoryLogger

# === Config ===
csv_path = "experiments/backtest_fragments/best_fragments_combined.csv"
symbol = "BTCUSDT"

# === Load data ===
df_fragments = pd.read_csv(csv_path, parse_dates=["timestamp"])
df_fragments.set_index("timestamp", inplace=True)
df_fragments.sort_index(inplace=True)

# === Load models ===
signal_model = SignalModel().load()
vol_model = VolatilityModel().load()
vol_features = vol_model.selected_features
sig_features = signal_model.selected_features

# === Result Summary ===
summary = []

# === Loop over each regime ===
for regime in df_fragments["regime_name"].unique():
    print(f"\n=== Evaluating Regime: {regime} ===")
    df = df_fragments[df_fragments["regime_name"] == regime].copy()
    df = prepare_features_only(df).dropna()
    print(f"[{regime}] Rows after feature prep: {len(df)}")

    if df.empty or len(df) < 10:
        print(f"[{regime}] Skipped due to insufficient rows after preprocessing.")
        continue

    signal_logger = SignalHistoryLogger()
    prediction_logger = PredictionLogger(autosave=False)

    for i in range(5, len(df) - 1):
        window = df.iloc[:i + 1]
        now = window.index[-2]

        vol_input = window.iloc[[-2]].reindex(columns=vol_features, fill_value=0.0)
        sig_input = window.iloc[[-2]].reindex(columns=sig_features, fill_value=0.0)

        try:
            vol_pred = vol_model.final_model.predict(vol_input)[0]
            if vol_pred != 1:
                # print(f"[{now}] {regime} → Skipped due to low volatility prediction")
                continue

            prob = signal_model.final_model.predict_proba(sig_input)[0][1]
            if prob >= 0.85:
                prediction = "UP"
            elif prob <= 0.15:
                prediction = "DOWN"
            else:
                # print(f"[{now}] {regime} → Skipped due to low confidence ({prob:.2%})")
                continue

            signal_type = "xgboost_bullish" if prediction == "UP" else "xgboost_bearish"
            conf_str = f"Conf={prob:.2%}" if prediction == "UP" else f"Conf={(1 - prob):.2%}"
            signal_logger.add_signal(signal_type, now, window["close"].iloc[-2], conf_str)

            prediction_logger.record_prediction(
                timestamp=now,
                prediction=prediction,
                close_now=window['close'].iloc[-1],
                close_prev=window['close'].iloc[-2],
                confidence=prob
            )

        except Exception as e:
            print(f"[{now}] {regime} → ERROR during prediction: {e}")
            continue

    df_result = prediction_logger.to_dataframe()
    num_preds = len(df_result)
    hit_rate = df_result["hit"].mean() if num_preds > 0 else 0.0

    print(f"[{regime}] Finished with {num_preds} predictions, hit rate: {hit_rate:.2%}")

    if num_preds > 0:
        summary.append({
            "regime": regime,
            "hit_rate": round(hit_rate * 100, 2),
            "num_predictions": num_preds
        })
    else:
        print(f"[{regime}] No confident predictions made.")

# === Final Summary ===
print("\n=== Backtest Summary ===")
if summary:
    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values(by="hit_rate", ascending=False).reset_index(drop=True)
    print(summary_df)
else:
    print("No valid predictions were made across all regimes.")

[Load] Model and features loaded.
[Load] Last trained: 2025-07-04T04:02:04.560046
[VolLoad] Model loaded with 4 features.
[VolLoad] Last trained: 2025-07-04T04:01:13.302181

=== Evaluating Regime: HighVol_DownTrend ===
[HighVol_DownTrend] Rows after feature prep: 501


C:\Users\PPH_Myat\OneDrive\Documents\GitHub\MMAT_signal_project\logs\signal_logger.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[HighVol_DownTrend] Finished with 68 predictions, hit rate: 39.71%

=== Evaluating Regime: HighVol_UpTrend ===
[HighVol_UpTrend] Rows after feature prep: 501


C:\Users\PPH_Myat\OneDrive\Documents\GitHub\MMAT_signal_project\logs\signal_logger.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[HighVol_UpTrend] Finished with 41 predictions, hit rate: 39.02%

=== Evaluating Regime: Sideways ===
[Sideways] Rows after feature prep: 501
[Sideways] Finished with 0 predictions, hit rate: 0.00%
[Sideways] No confident predictions made.

=== Evaluating Regime: LowVol_DownTrend ===
[LowVol_DownTrend] Rows after feature prep: 501


C:\Users\PPH_Myat\OneDrive\Documents\GitHub\MMAT_signal_project\logs\signal_logger.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[LowVol_DownTrend] Finished with 2 predictions, hit rate: 100.00%

=== Evaluating Regime: LowVol_UpTrend ===
[LowVol_UpTrend] Rows after feature prep: 501
[LowVol_UpTrend] Finished with 0 predictions, hit rate: 0.00%
[LowVol_UpTrend] No confident predictions made.

=== Backtest Summary ===
              regime  hit_rate  num_predictions
0   LowVol_DownTrend    100.00                2
1  HighVol_DownTrend     39.71               68
2    HighVol_UpTrend     39.02               41


In [5]:
import pandas as pd
from signals.model_preparation import prepare_features_only
from train_model.signal_model import SignalModel
from train_model.vol_model import VolatilityModel
from logs.prediction_logger import PredictionLogger
from logs.signal_logger import SignalHistoryLogger

# === Config ===
csv_path = "experiments/backtest_fragments/best_fragments_combined.csv"
symbol = "BTCUSDT"

# === Load data ===
df_fragments = pd.read_csv(csv_path, parse_dates=["timestamp"])
df_fragments.set_index("timestamp", inplace=True)
df_fragments.sort_index(inplace=True)

# === Load models ===
signal_model = SignalModel().load()
vol_model = VolatilityModel().load()
vol_features = vol_model.selected_features
sig_features = signal_model.selected_features

# === Result Summary ===
summary = []

# === Loop over each regime ===
for regime in df_fragments["regime_name"].unique():
    print(f"\n=== Evaluating Regime: {regime} ===")
    df = df_fragments[df_fragments["regime_name"] == regime].copy()
    df = prepare_features_only(df).dropna()
    print(f"[{regime}] Rows after feature prep: {len(df)}")

    if df.empty or len(df) < 10:
        print(f"[{regime}] Skipped due to insufficient rows after preprocessing.")
        continue

    signal_logger = SignalHistoryLogger()
    prediction_logger = PredictionLogger(autosave=False)

    for i in range(5, len(df) - 1):
        window = df.iloc[:i + 1]
        now = window.index[-2]

        vol_input = window.iloc[[-2]].reindex(columns=vol_features, fill_value=0.0)
        sig_input = window.iloc[[-2]].reindex(columns=sig_features, fill_value=0.0)

        try:
            vol_pred = vol_model.final_model.predict(vol_input)[0]
            if vol_pred != 1:
                # print(f"[{now}] {regime} → Skipped due to low volatility prediction")

                continue

            prob = signal_model.final_model.predict_proba(sig_input)[0][1]
            prediction = "UP" if prob >= 0.5 else "DOWN"

            signal_type = "xgboost_bullish" if prediction == "UP" else "xgboost_bearish"
            conf_str = f"Conf={prob:.2%}" if prediction == "UP" else f"Conf={(1 - prob):.2%}"
            signal_logger.add_signal(signal_type, now, window["close"].iloc[-2], conf_str)

            prediction_logger.record_prediction(
                timestamp=now,
                prediction=prediction,
                close_now=window['close'].iloc[-1],
                close_prev=window['close'].iloc[-2],
                confidence=prob
            )

        except Exception as e:
            print(f"[{now}] {regime} → ERROR during prediction: {e}")
            continue

    df_result = prediction_logger.to_dataframe()
    num_preds = len(df_result)
    hit_rate = df_result["hit"].mean() if num_preds > 0 else 0.0

    print(f"[{regime}] Finished with {num_preds} predictions, hit rate: {hit_rate:.2%}")

    if num_preds > 0:
        summary.append({
            "regime": regime,
            "hit_rate": round(hit_rate * 100, 2),
            "num_predictions": num_preds
        })
    else:
        print(f"[{regime}] No predictions made.")

# === Final Summary ===
print("\n=== Backtest Summary ===")
if summary:
    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values(by="hit_rate", ascending=False).reset_index(drop=True)
    print(summary_df)
else:
    print("No valid predictions were made across all regimes.")

[Load] Model and features loaded.
[Load] Last trained: 2025-07-04T04:02:04.560046
[VolLoad] Model loaded with 4 features.
[VolLoad] Last trained: 2025-07-04T04:01:13.302181

=== Evaluating Regime: HighVol_DownTrend ===
[HighVol_DownTrend] Rows after feature prep: 501


C:\Users\PPH_Myat\OneDrive\Documents\GitHub\MMAT_signal_project\logs\signal_logger.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[HighVol_DownTrend] Finished with 495 predictions, hit rate: 44.24%

=== Evaluating Regime: HighVol_UpTrend ===
[HighVol_UpTrend] Rows after feature prep: 501


C:\Users\PPH_Myat\OneDrive\Documents\GitHub\MMAT_signal_project\logs\signal_logger.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[HighVol_UpTrend] Finished with 495 predictions, hit rate: 43.23%

=== Evaluating Regime: Sideways ===
[Sideways] Rows after feature prep: 501
[Sideways] Finished with 0 predictions, hit rate: 0.00%
[Sideways] No predictions made.

=== Evaluating Regime: LowVol_DownTrend ===
[LowVol_DownTrend] Rows after feature prep: 501


C:\Users\PPH_Myat\OneDrive\Documents\GitHub\MMAT_signal_project\logs\signal_logger.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[LowVol_DownTrend] Finished with 2 predictions, hit rate: 100.00%

=== Evaluating Regime: LowVol_UpTrend ===
[LowVol_UpTrend] Rows after feature prep: 501
[LowVol_UpTrend] Finished with 0 predictions, hit rate: 0.00%
[LowVol_UpTrend] No predictions made.

=== Backtest Summary ===
              regime  hit_rate  num_predictions
0   LowVol_DownTrend    100.00                2
1  HighVol_DownTrend     44.24              495
2    HighVol_UpTrend     43.23              495


In [6]:
import pandas as pd
from signals.model_preparation import prepare_features_only
from train_model.signal_model import SignalModel
from train_model.vol_model import VolatilityModel
from logs.prediction_logger import PredictionLogger
from logs.signal_logger import SignalHistoryLogger

# === Config ===
csv_path = "experiments/backtest_fragments/best_fragments_combined.csv"
symbol = "BTCUSDT"

# === Load data ===
df_fragments = pd.read_csv(csv_path, parse_dates=["timestamp"])
df_fragments.set_index("timestamp", inplace=True)
df_fragments.sort_index(inplace=True)

# === Load models ===
signal_model = SignalModel().load()
vol_model = VolatilityModel().load()
vol_features = vol_model.selected_features
sig_features = signal_model.selected_features

# === Result Summary ===
summary = []

# === Loop over each regime ===
for regime in df_fragments["regime_name"].unique():
    print(f"\n=== Evaluating Regime: {regime} ===")
    df = df_fragments[df_fragments["regime_name"] == regime].copy()
    df = prepare_features_only(df).dropna()
    print(f"[{regime}] Rows after feature prep: {len(df)}")

    if df.empty or len(df) < 10:
        print(f"[{regime}] Skipped due to insufficient rows after preprocessing.")
        continue

    signal_logger = SignalHistoryLogger()
    prediction_logger = PredictionLogger(autosave=False)

    for i in range(5, len(df) - 1):
        window = df.iloc[:i + 1]
        now = window.index[-2]

        # Skip vol filter — just compute inputs but don't condition on it
        vol_input = window.iloc[[-2]].reindex(columns=vol_features, fill_value=0.0)
        sig_input = window.iloc[[-2]].reindex(columns=sig_features, fill_value=0.0)

        try:
            prob = signal_model.final_model.predict_proba(sig_input)[0][1]
            if prob >= 0.85:
                prediction = "UP"
            elif prob <= 0.15:
                prediction = "DOWN"
            else:
                continue  # confidence too low

            signal_type = "xgboost_bullish" if prediction == "UP" else "xgboost_bearish"
            conf_str = f"Conf={prob:.2%}" if prediction == "UP" else f"Conf={(1 - prob):.2%}"
            signal_logger.add_signal(signal_type, now, window["close"].iloc[-2], conf_str)

            prediction_logger.record_prediction(
                timestamp=now,
                prediction=prediction,
                close_now=window['close'].iloc[-1],
                close_prev=window['close'].iloc[-2],
                confidence=prob
            )

        except Exception as e:
            print(f"[{now}] {regime} → ERROR during prediction: {e}")
            continue

    df_result = prediction_logger.to_dataframe()
    num_preds = len(df_result)
    hit_rate = df_result["hit"].mean() if num_preds > 0 else 0.0

    print(f"[{regime}] Finished with {num_preds} predictions, hit rate: {hit_rate:.2%}")

    if num_preds > 0:
        summary.append({
            "regime": regime,
            "hit_rate": round(hit_rate * 100, 2),
            "num_predictions": num_preds
        })
    else:
        print(f"[{regime}] No confident predictions made.")

# === Final Summary ===
print("\n=== Backtest Summary ===")
if summary:
    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values(by="hit_rate", ascending=False).reset_index(drop=True)
    print(summary_df)
else:
    print("No valid predictions were made across all regimes.")


[Load] Model and features loaded.
[Load] Last trained: 2025-07-04T04:02:04.560046
[VolLoad] Model loaded with 4 features.
[VolLoad] Last trained: 2025-07-04T04:01:13.302181

=== Evaluating Regime: HighVol_DownTrend ===
[HighVol_DownTrend] Rows after feature prep: 501


C:\Users\PPH_Myat\OneDrive\Documents\GitHub\MMAT_signal_project\logs\signal_logger.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[HighVol_DownTrend] Finished with 68 predictions, hit rate: 39.71%

=== Evaluating Regime: HighVol_UpTrend ===
[HighVol_UpTrend] Rows after feature prep: 501


C:\Users\PPH_Myat\OneDrive\Documents\GitHub\MMAT_signal_project\logs\signal_logger.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[HighVol_UpTrend] Finished with 41 predictions, hit rate: 39.02%

=== Evaluating Regime: Sideways ===
[Sideways] Rows after feature prep: 501


...

[Sideways] Finished with 128 predictions, hit rate: 41.41%

=== Evaluating Regime: LowVol_DownTrend ===
[LowVol_DownTrend] Rows after feature prep: 501


C:\Users\PPH_Myat\OneDrive\Documents\GitHub\MMAT_signal_project\logs\signal_logger.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[LowVol_DownTrend] Finished with 61 predictions, hit rate: 59.02%

=== Evaluating Regime: LowVol_UpTrend ===
[LowVol_UpTrend] Rows after feature prep: 501


C:\Users\PPH_Myat\OneDrive\Documents\GitHub\MMAT_signal_project\logs\signal_logger.py:41: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[LowVol_UpTrend] Finished with 116 predictions, hit rate: 50.86%

=== Backtest Summary ===
              regime  hit_rate  num_predictions
0   LowVol_DownTrend     59.02               61
1     LowVol_UpTrend     50.86              116
2           Sideways     41.41              128
3  HighVol_DownTrend     39.71               68
4    HighVol_UpTrend     39.02               41
